**Poetry Generator (RNN Markov)**

This scipt uses Recurrent Neural Networks and Markov Chains in order to generate new verses in the style of the poems that it is given as input.  The Markovify functions (Markov Chains) are used to build new sentences (based off of word1=>word2 probabilities), while the Keras.LSTM functions (Recurrent Neural Networks) are used to predict the properties of the next line of the poem (e.g. # syllables, rhyme scheme) such that an appropriate new sentence can be selected (from the setences that were generated with Markovify).  This approach is described in the following publications ([Link #1](http://www.emnlp2015.org/proceedings/EMNLP/pdf/EMNLP221.pdf), [Link #2](https://arxiv.org/pdf/1612.03205.pdf)) and many of the functions were adapted from the following script ([Link #3](https://github.com/robbiebarrat/rapping-neural-network/blob/master/model.py)).  Currently this approach does a better job at making sentences that rhyme as opposed to making sentences that are logical and as such it is best used to generate nonsensical rap lyrics similar to this verse by hip-hop artist Riff Raff:
> Spandex (spandex), I pull up with a lamb text --
> Too strudel, toaster strudel, ballin' on you poodles --
> Swish! Two cougars, 40 plus, clutching Rugers --
> Got 'em running over Jerome Bettis, Versace lettuce --
> No dieting, Gucci eyelids, I go to sleep, snobby pilots --

Here we will generate new poems using the vocabulary and rhyme structure of both Notorious B.I.G. and Lil Wayne.

Here are the first 1000 characters from the collection of poems by Notorious B.I.G.

In [0]:
artist_file = 'fs.txt'
with open(artist_file) as f: # The with keyword automatically closes the file when you are done
    print (f.read(1000))

﻿Rapea, te gano sacowea
Yo voy a ser el que en el micrófono lo pesca y te pasea
Estoy haciendo rap, este pollo, yo me distingo
Criatura, dime qué se siente estar en el olimpo
¿Qué se siente?, no voy a insultarte
La batallas buenas, cabros siempre tienen segunda parte
Hoy día voy a condenarte
No eres el que me pasea, eres el más llorón de los participantes
Ah, de los participantes
Yo estoy haciendo el rap porque con esto tengo el desplante
Es lo que yo hago con esto, es un pacto eterno
Yo te condeno porque soy Hades en el infierno
Eres Hades en el infierno, eso no me complica
Porque este weón todo lo sataniza
Estabai llorando como el (...)
Siéntente contento, si entraste a esta weá por pura chiripa
Ay, dime que como mi mente no canaliza
Vo' soi entero malo, aweonao, aterriza
Yo estoy haciendo hip hop a mi estilo, se divisa
Eneese a este weón le pega una paliza
Paliza, no puede hacer más
Si es que puedo demostrar nivel de la ciudad
Concepción, en verdad, yo te voy a enseñar
Mejor (...) q

Here are the first 1000 characters from the collection of poems by Lil Wayne

Plot word frequencies for .txt files

Here we use Recurrent Neural Networks and Markov chains to generate new lyrics in the style of the input text.
The Markovify functions (Markov Chains) are used to build new sentences (based off of word1=>word2 probabilities), while the Keras.LSTM functions (Recurrent Neural Networks) are used to predict the properties of the next line of the poem (e.g. # syllables, rhyme scheme), such that an appropriate new sentence can be selected (from the setences that were generated with Markovify).

In [0]:
!pip install pronouncing
!pip install markovify
!pip install tqdm
import pronouncing
import markovify
import re
import random
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import LSTM 
from keras.layers.core import Dense

Recurrent Neural Network (https://keras.io/layers/recurrent/#lstm)

In [0]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))
	model.add(LSTM(2, return_sequences=True))
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

Markov Chain (https://github.com/jsvine/markovify)

In [0]:
def markov(text_file):
    ######
	read = open(text_file, "r", encoding='utf-8').read()
	text_model = markovify.NewlineText(read)
	return text_model

Determine number of syllables in line

In [0]:
def syllables(line):
	count = 0
	for word in line.split(" "):
		vowels = 'aeiouy'
# 		word = word.lower().strip("!@#$%^&*()_+-={}[];:,.<>/?")
		word = word.lower().strip(".:;?!")
		if word == '':
			continue
		if word[0] in vowels:
			count +=1
		for index in range(1,len(word)):
			if word[index] in vowels and word[index-1] not in vowels:
				count +=1
		if word.endswith('e'):
			count -= 1
		if word.endswith('le'):
			count+=1
		if count == 0:
			count +=1
	return count / maxsyllables

Make index of words that rhyme with your word

In [0]:
def rhymeindex(lyrics):
	if str(artist) + ".rhymes" in os.listdir(".") and train_mode == False:
		print ("loading saved rhymes from " + str(artist) + ".rhymes")
		return open(str(artist) + ".rhymes", "r",encoding='utf-8').read().split("\n")
	else:
		rhyme_master_list = []
		print ("Building list of rhymes:")
		for i in lyrics:
			word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()
			rhymeslist = pronouncing.rhymes(word)
			rhymeslistends = []      
			for i in rhymeslist:
				rhymeslistends.append(i[-2:])
			try:
				rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
			except Exception:
				rhymescheme = word[-2:]
			rhyme_master_list.append(rhymescheme)
		rhyme_master_list = list(set(rhyme_master_list))
		reverselist = [x[::-1] for x in rhyme_master_list]
		reverselist = sorted(reverselist)
		rhymelist = [x[::-1] for x in reverselist]
		print("List of Sorted 2-Letter Rhyme Ends:")
		print(rhymelist)
		f = open(str(artist) + ".rhymes", "w", encoding='utf-8')
		f.write("\n".join(rhymelist))
		f.close()
		return rhymelist

Make index of rhymes that you use

In [0]:
def rhyme(line, rhyme_list):
	word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()
	rhymeslist = pronouncing.rhymes(word)
	rhymeslistends = []
	for i in rhymeslist:
		rhymeslistends.append(i[-2:])
	try:
		rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
	except Exception:
		rhymescheme = word[-2:]
	try:
		float_rhyme = rhyme_list.index(rhymescheme)
		float_rhyme = float_rhyme / float(len(rhyme_list))
		return float_rhyme
	except Exception:
		float_rhyme = None
		return float_rhyme

Separate each line of the input txt

In [0]:
def split_lyrics_file(text_file):
	text = open(text_file, encoding='utf-8').read()
	text = text.split("\n")
	while "" in text:
		text.remove("")
	return text

Generate lyrics

In [0]:
from tqdm import tqdm
from math import ceil

def get_last_word(bar):
	last_word = bar.split(" ")[-1]
	if last_word[-1] in "!.?,":
		last_word = last_word[:-1]
	return last_word

def generate_lyrics(text_model, text_file, wanted_word):
	bars = []
	last_words = []
	lyriclength = len(open(text_file,encoding='utf-8').read().split("\n"))
	markov_model = markov(text_file)
 
	rhymes_amount = 500 #lyriclength / 9
	pbar = tqdm(total=ceil(rhymes_amount),position=0,leave=True)
	while len(bars) < rhymes_amount:
		if (len(bars) % 32) == 0:
			bar = markov_model.make_sentence_with_start(wanted_word, strict=False, tries=100)
		else:
			bar = markov_model.make_sentence(max_overlap_ratio = .49, tries=100)
		if type(bar) != type(None) and syllables(bar) < 1:
			last_word = get_last_word(bar)
			if bar not in bars and last_words.count(last_word) < 3:
				bars.append(bar)
				last_words.append(last_word)
				pbar.update()
	pbar.close()
	return bars

Build dataset

In [0]:
def build_dataset(lines, rhyme_list):
	dataset = []
	line_list = []
	for line in lines:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	x_data = []
	y_data = []
	for i in range(len(dataset) - 3):
		line1 = dataset[i    ][1:]
		line2 = dataset[i + 1][1:]
		line3 = dataset[i + 2][1:]
		line4 = dataset[i + 3][1:]
		x = [line1[0], line1[1], line2[0], line2[1]]
		x = np.array(x)
		x = x.reshape(2,2)
		x_data.append(x)
		y = [line3[0], line3[1], line4[0], line4[1]]
		y = np.array(y)
		y = y.reshape(2,2)
		y_data.append(y)
	x_data = np.array(x_data)
	y_data = np.array(y_data)
	return x_data, y_data

Compose verse

In [0]:
def compose_rap(lines, rhyme_list, lyrics_file, model):
	rap_vectors = []
	human_lyrics = split_lyrics_file(lyrics_file)
	initial_index = random.choice(range(len(human_lyrics) - 1))
	initial_lines = human_lyrics[initial_index:initial_index + 2]
	starting_input = []
	for line in initial_lines:
		starting_input.append([syllables(line), rhyme(line, rhyme_list)])
	starting_vectors = model.predict(np.array([starting_input]).flatten().reshape(1, 2, 2))
	rap_vectors.append(starting_vectors)
	for i in range(50):
		rap_vectors.append(model.predict(np.array([rap_vectors[-1]]).flatten().reshape(1, 2, 2)))
	return rap_vectors

Compose verse (part 2)

In [0]:
#Auxiliary methods

def last_word_compare(rap, line2):
	penalty = 0
	for line1 in rap:
		word1 = line1.split(" ")[-1]
		word2 = line2.split(" ")[-1]
		while word1[-1] in "?!,. ":
			word1 = word1[:-1]
		while word2[-1] in "?!,. ":
			word2 = word2[:-1]
		if word1 == word2:
			penalty += 0.2
	return penalty

def calculate_score(vector_half, syllables, rhyme, penalty, rhyme_list):
	desired_syllables = vector_half[0]
	desired_rhyme = vector_half[1]
	desired_syllables = desired_syllables * maxsyllables
	desired_rhyme = desired_rhyme * len(rhyme_list)
	score = 1.0 - abs(float(desired_syllables) - float(syllables)) + abs(float(desired_rhyme) - float(rhyme)) - penalty
	return score

# Main method

def vectors_into_song(vectors, generated_lyrics, rhyme_list, wanted_word):
	print()	
	print("Escribiendo un verso con:", wanted_word)
	print()
	dataset = []
	for line in generated_lyrics:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	rap = []
	vector_halves = []
	for vector in vectors:
		vector_halves.append(list(vector[0][0])) 
		vector_halves.append(list(vector[0][1]))
	for vector in vector_halves:
		scorelist = []
		for item in dataset:
			line = item[0]
			if len(rap) != 0:
				penalty = last_word_compare(rap, line)
			else:
				penalty = 0
			total_score = calculate_score(vector, item[1], item[2], penalty, rhyme_list)
			score_entry = [line, total_score]
			scorelist.append(score_entry)
		fixed_score_list = [0]
		for score in scorelist:
			fixed_score_list.append(float(score[1]))
		max_score = max(fixed_score_list)
		for item in scorelist:
			if item[1] == max_score:
				rap.append(item[0])
				#print (str(item[0]))
				for i in dataset:
					if item[0] == i[0]:
						dataset.remove(i)
						break
				break
	f = open(rap_file, "w", encoding='utf-8')
	bar_index=0            
	while bar_index < len(rap):
		first = rap[bar_index].split(" ")[0]
		if first == wanted_word:
			for bar_index in range(bar_index, bar_index+4):
				print(rap[bar_index])                
				f.write(rap[bar_index])
				f.write("\n")
			f.write("\n")
			return rap
		else:
			bar_index+=1
	return rap

Traning function

In [0]:
def train(x_data, y_data, model):
	model.fit(np.array(x_data), np.array(y_data),
			  batch_size=2,
			  epochs=5,
			  verbose=1)
	model.save_weights(artist + ".rap")

Function to train the model

In [0]:
def train_model(model, text_model, depth):
	bars = split_lyrics_file(text_file)
	rhyme_list = rhymeindex(bars)
	x_data, y_data = build_dataset(bars, rhyme_list)
	train(x_data, y_data, model)

Function to run the model

In [0]:
def run_model(model, text_model, depth, wanted_word):
  print("Generando rimas usando", wanted_word)
  bars = generate_lyrics(text_model, text_file, wanted_word)
  rhyme_list = rhymeindex(bars)
  print("Componiendo verso usando", wanted_word)
  vectors = compose_rap(bars, rhyme_list, text_file, model)
  rap = vectors_into_song(vectors, bars, rhyme_list, wanted_word)

Some important parameters to keep in mind are as follows: (1) maxsyllables: max # of syllables per line; (2) max_overlap_ratio: how different the output should be from the original input; (3) tries: how many times to try to build a line that satisifies some parameter; (4) epochs: how many times for the NN to pass over the data.

To reduce computational complexity, maximize (1) and (2) and minimize (3) and (4).  
Computational time will increase dramatically when (1) is less than 8 and when (2) is less than 0.5

In [0]:
depth = 4 
maxsyllables = 8
artist = "artist"
rap_file = "temporary_poem.txt"

Train the model using *text_file*

In [0]:
maxsyllables = 8
text_file = 'concatenated.txt'
model = create_network(depth)
text_model = markov(text_file)    
train_model(model, text_model, depth)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_97 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_98 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_99 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_100 (LSTM)              (None, 2, 8)              544       
_________________________________________________________________
lstm_101 (LSTM)              (None, 2, 8)              544       
_________________________________________________________________
lstm_102 (LSTM)              (None, 2, 2)              88        
Total params: 2,248
Trainable params: 2,248
Non-trainable params: 0
___________________________________________________

Correr el modelo en funcion de la palabra ingresada

In [0]:
word = input("Ingrese una palabra:")
print()
while word != '':
  run_model(model, text_model, depth, word)
  print()
  word = input("Ingrese una palabra:")


Generando rimas usando Estoy


100%|██████████| 500/500 [53:14<00:00,  6.39s/it]


loading saved rhymes from artist.rhymes
Componiendo verso usando Estoy

Escribiendo un verso con: Estoy



TypeError: ignored

To Do: (1) Add More Pre-Processing Steps; (2) Increase Size of Individual Lyric Files;